In [1]:
import numpy as np

In [4]:
arr = np.array([
    [1, 2, 3],
    [3, 4, 5],
    [5, 6, 7]
])

In [5]:
np.roll(arr, 1, axis=1)

array([[3, 1, 2],
       [5, 3, 4],
       [7, 5, 6]])

In [6]:
np.roll(arr, -1, axis=1)

array([[2, 3, 1],
       [4, 5, 3],
       [6, 7, 5]])

In [7]:
t = np.roll(arr, 1, axis=1)
np.roll(t, -1, axis=1)

array([[1, 2, 3],
       [3, 4, 5],
       [5, 6, 7]])

In [8]:
t = np.roll(arr, 2, axis=1)
np.roll(t, -2, axis=1)

array([[1, 2, 3],
       [3, 4, 5],
       [5, 6, 7]])

In [1]:
class Action:
    
    def __init__(self, func):
        self.function = func
        
    def run(self, state, params):
        return self.function(state, params)

In [31]:
class Scenario:
    
    def __init__(self, actions):
        self.actions = actions
        
    def run(self, state, params):
        for action in self.actions:
            state = action.run(state, params)
        
        return state

In [32]:
def interpolate_int(start, end, steps):
    return list(np.linspace(start, end, steps+2).astype(int))

def interpolate_float(start, end, steps):
    return list(np.linspace(start, end, steps+2))

def interpolate_str(start, end, steps):
    first = steps // 2
    return [start,] * (first + 1) + [end,] * (steps - first + 1)

def interpolate_vec(start, end, steps):
    raise Exception('Not implemented error')

In [33]:
def test_interp():
    s = 1
    e = 10
    i = interpolate_int(s, e, 8)
    print(i)
    assert i == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    s = 1
    e = 2
    i = interpolate_float(s, e, 1)
    print(i)
    assert i == [1, 1.5, 2]
    
    s = 'first'
    e = 'second'
    i = interpolate_str(s, e, 3)
    print(i)
    assert i == ['first',] * 2 + ['second',] * 3

In [34]:
test_interp()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1.0, 1.5, 2.0]
['first', 'first', 'second', 'second', 'second']


In [56]:
class ScenarioRunner:
    
    def __init__(self, scenario):
        self.scenario = scenario
        
    def run(self, init_params, end_params, init_state, steps):
        params_generator = Interpolator.interp(init_params, end_params, steps)
        
        state = init_state.copy()
        for params in params_generator:
            print(f'{state=}')
            print(f'{params=}')
            state = self.scenario.run(state, params)
        
        return state

In [57]:
class Interpolator:
    
    def interp(start, end, steps):
        yield start
        
        all_states_dict = {}
        
        for k in start.keys():
            vs = start[k]
            ve = end[k]
            assert type(vs) == type(ve), 'Type of start and end values must be same'
            
            if vs == ve:
                interps = [vs,] * (steps + 2) 
            # TODO: test with different int types
            if type(vs) is int:
                interps = interpolate_int(vs, ve, steps)
            elif type(vs) is float:
                interps = interpolate_float(vs, ve, steps)
            elif type(vs) is str:
                interps = interpolate_str(vs, ve, steps)
            else:
                raise Exception('Not implemented error')

            all_states_dict[k] = interps
        
        print('All different states:')
        print(all_states_dict)
        
        state = start.copy()
        for i in range(steps):
            for k in state.keys():
                state[k] = all_states_dict[k][i+1]
            
            yield state
        
        yield end

In [60]:
def func_1(state: dict, params: dict):
    state['x'] = state['result']
    state['y'] = state['x'] * params['a']
    return state

def func_2(state: dict, params: dict):
    state['result'] = state['x'] + state['y'] + params['b']
    return state

In [61]:
action1 = Action(func_1)
action2 = Action(func_2)

init_params = {
    'a': 5,
    'b': 10,
}

end_params = {
    'a': 10,
    'b': 1,
}

init_state = {
    'x': 0,
    'y': 0,
    'result': 0
}

steps = 3

scenario = Scenario([action1, action2])

scenario_runner = ScenarioRunner(scenario)

result_state = scenario_runner.run(init_params, end_params, init_state, steps)

state={'x': 0, 'y': 0, 'result': 0}
params={'a': 5, 'b': 10}
All different states:
{'a': [5, 6, 7, 8, 10], 'b': [10, 7, 5, 3, 1]}
state={'x': 0, 'y': 0, 'result': 10}
params={'a': 6, 'b': 7}
state={'x': 10, 'y': 60, 'result': 77}
params={'a': 7, 'b': 5}
state={'x': 77, 'y': 539, 'result': 621}
params={'a': 8, 'b': 3}
state={'x': 621, 'y': 4968, 'result': 5592}
params={'a': 10, 'b': 1}


In [62]:
result_state

{'x': 5592, 'y': 55920, 'result': 61513}

In [63]:
!git clone https://github.com/assafshocher/ResizeRight.git

Cloning into 'ResizeRight'...
